In [12]:
import json
import re

In [2]:
isotopes = {}

with open('../element_data_with_decays.json') as f:
    isotopes = json.load(f)

In [3]:
len(isotopes)

3143

In [21]:
remove_metastable = re.compile(r'(\d)(m)')

In [22]:
remove_metastable.sub(r'\1', '229mPa')

'229Pa'

In [27]:
def process_products(decay_products):
    decay_output = {}
    if decay_products is None:
        return decay_output
    for product in decay_products:
        if not product['nucleide']:
            continue
        individual_products = product['nucleide'].split('\n')
        for product in individual_products:
            product = remove_metastable.sub(r'\1', product)
            if product and (product != '(various)'):
                decay_output[product] = 1
    return decay_output
            
        
first_order_products = {isotope['nuclide_symbol']: process_products(isotope['decay']) for isotope in isotopes}

In [80]:
first_order_products

{'25Na': {'25Mg': 1},
 '71Br': {'71Se': 1},
 '40S': {'40Cl': 1},
 '151Gd': {'147Sm': 1, '151Eu': 1},
 '81Kr': {'81Br': 1},
 '208Ac': {'204Fr': 1, '208Ac': 1, '208Ra': 1},
 '215Bi': {'215Po': 1},
 '157Pm': {'157Sm': 1},
 '200Pt': {'200Au': 1},
 '99Mo': {'99Tc': 1},
 '24Al': {'20Ne': 1, '23Na': 1, '24Mg': 1},
 '53Cu': {'52Ni': 1},
 '41K': {},
 '76Ge': {'76Se': 1},
 '157Gd': {},
 '71Rb': {'70Kr': 1},
 '105Y': {'105Zr': 1},
 '188Hg': {'184Pt': 1, '188Au': 1},
 '20F': {'20Ne': 1},
 '107Tc': {'107Ru': 1},
 '226U': {'222Th': 1},
 '237Cf': {'233Cm': 1, '237Bk': 1},
 '102Mo': {'102Tc': 1},
 '84Mo': {'84Nb': 1},
 '245Es': {'241Bk': 1, '245Cf': 1},
 '60Mn': {'60Fe': 1, '60Mn': 1},
 '77Rb': {'77Kr': 1},
 '200Fr': {'196At': 1, '200Fr': 1},
 '266Bh': {'262Db': 1},
 '200At': {'196Bi': 1, '200At': 1, '200Po': 1},
 '139Te': {'138I': 1, '139I': 1},
 '135La': {'135Ba': 1},
 '35Ca': {'34Ar': 1, '35K': 1},
 '105Te': {},
 '227Pa': {'223Ac': 1, '227Th': 1},
 '259Rf': {'255No': 1, '259Lr': 1},
 '190Ir': {'190

In [104]:
def further_decay(isotope, depth = 1, decay_chain = None):
    output = {}
    decay_chain = set([isotope])
    for decay_product in first_order_products[isotope]:
        # Avoid any cycles (though these shouldn't exist)
        if decay_product in decay_chain or depth > 20:
            return output
        decay_chain.update([decay_product])
        if decay_product in first_order_products:
            output[decay_product] = depth
            output.update(further_decay(decay_product, depth + 1, decay_chain))
    return output 

In [105]:
nth_order_decay_2 = {isotope: further_decay(isotope) for isotope in first_order_products}

In [106]:
nth_order_decay_2

{'25Na': {'25Mg': 1},
 '71Br': {'71As': 2, '71Ga': 4, '71Ge': 3, '71Se': 1},
 '40S': {'40Ar': 2, '40Cl': 1},
 '159Pr': {'159Eu': 4,
  '159Gd': 5,
  '159Nd': 1,
  '159Pm': 2,
  '159Sm': 3,
  '159Tb': 6},
 '81Kr': {'81Br': 1},
 '81Zr': {'80Kr': 3,
  '80Rb': 2,
  '80Sr': 1,
  '81Br': 5,
  '81Kr': 4,
  '81Rb': 3,
  '81Sr': 2,
  '81Y': 1},
 '157Pm': {'157Eu': 2, '157Gd': 3, '157Sm': 1},
 '200Pt': {'200Au': 1, '200Hg': 2},
 '99Mo': {'99Ru': 2, '99Tc': 1},
 '24Al': {'20Ne': 1, '23Na': 1, '24Mg': 1},
 '53Cu': {'51Cr': 4, '51Fe': 2, '51Mn': 3, '51V': 5, '52Co': 2, '52Ni': 1},
 '41K': {},
 '76Ge': {'76Se': 1},
 '157Gd': {},
 '71Rb': {'70Br': 2, '70Kr': 1},
 '105Y': {'103Mo': 3,
  '103Rh': 6,
  '103Ru': 5,
  '103Tc': 4,
  '104Mo': 3,
  '104Nb': 2,
  '104Ru': 5,
  '104Tc': 4,
  '105Mo': 3,
  '105Nb': 2,
  '105Pd': 7,
  '105Rh': 6,
  '105Ru': 5,
  '105Tc': 4,
  '105Zr': 1},
 '107Ru': {'107Ag': 3, '107Pd': 2, '107Rh': 1},
 '20F': {'20Ne': 1},
 '107Tc': {'107Ag': 4, '107Pd': 3, '107Rh': 2, '107Ru': 1

In [107]:
with open('../decay_chains.json', 'w') as f:
    json.dump(nth_order_decay_2, f, indent=4, separators=(',', ':'))

In [31]:
z= {**{'a':3, 'b':4}, **{'c':3, 'd':4}}

{'a': 3, 'b': 4, 'c': 3, 'd': 4}